## 참조자료

https://myungpyo.medium.com/reading-coroutine-official-guide-thoroughly-part-1-98f6e792bd5b

## 코루티 모듈 로딩 

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.1")

In [2]:
:classpath

Current classpath (14 paths):
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/lib-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/api-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.7.0-dev-1825.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.6.0.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx/kotlinx-coroutines-core/1.6.1/kotlinx-coroutines-core-1.6.1.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx

In [3]:
import kotlinx.coroutines.*

## 설치 확인 

## 기본 사용 패키지

## 코루틴 클래스 확인 

In [4]:
println((GlobalScope::class).supertypes)             // 수퍼 클래스 확인 

println(GlobalScope === GlobalScope)                 // 글로벌 스코프는 object 선언의 싱글턴

println(GlobalScope::class.isInstance(GlobalScope))  // 인스턴스 관계 확인 

val co = CoroutineScope(Dispatchers.Default)         // 코루틴 스코프는 클래스

println(CoroutineScope::class.supertypes)            // 수퍼클래스 확인 

println(CoroutineScope::class.isInstance(co))        // 인스턴스 관계 확인 

[kotlinx.coroutines.CoroutineScope, kotlin.Any]
true
true
[kotlin.Any]
true


## 일반함수 호출과 코루틴처리 비교

In [5]:
fun task1()= println("일반함수 task 1 : " 
                     + Thread.currentThread().name)       // 첫번째 함수 
fun task2() = println("일반함수 task 2 : "                  // 두번째 함수 
                     + Thread.currentThread().name)

run {  println("순차적 처리")                                // 내부 함수를 연속적으로 실행 
    task1();task2();  }                                   // 일시중단함수 사용금지  

println("스레드 처리")
val thread1 = object : Thread() {                         // 스레드 처리
      override fun run(){                                 // start 메소드 이후에 run 실행         
        task1()                                           // 
        Thread.sleep(200)                                 // 스레드 일시중단 
      }                                                   
}

val thread2 = object : Thread() {                         // 다른 스레드 처리
      override fun run(){                                 // start 메소드 이후에 run 실행         
        task2()                                           // 
        Thread.sleep(200)                                 // 스레드 일시중단 
      }                                                   
}
thread1.start()  
thread2.start()                                           // 스레드 시작
thread2.join()
thread1.join()

순차적 처리
일반함수 task 1 : Thread-18
일반함수 task 2 : Thread-18
스레드 처리
일반함수 task 1 : Thread-19
일반함수 task 2 : Thread-20


## 코루틴 처리하기

In [6]:
import kotlinx.coroutines.*                               // 코루틴 패키지

fun task1()= println("일반함수 task 1 : " 
                     + Thread.currentThread().name)       // 첫번째 함수 
fun task2() = println("일반함수 task 2 : "                  // 두번째 함수 
                     + Thread.currentThread().name)

suspend fun task3() =  println("일수중단함수 task 3 : "      // 일시중단함수 정의
                     + Thread.currentThread().name)
val task4 = suspend {println("일시중단함수 task 4 : "        // 람다표현식도 일시중단함수 정의 가능 
                     + Thread.currentThread().name)}

val rb = runBlocking { 
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
    launch {                                              // 코루틴 빌더로 함수처리
        delay(300)
        task1()                                           // 일반함수 호출 
    }
    launch {                                              // 코루틴 빌더로 함수처리
        task2()                                           // 일반함수 호출 
        delay(300)
    }
    launch {                                              // 코루틴 빌더로 함수처리
        delay(300)
        task3()                                           // 일시중단함수 
    }
    launch {                                              // 코루틴 빌더로 함수처리
        task4()                                           // 일시중단함수
        delay(300)
    }
}                                                         // 메인 스레드 사용

코루틴 처리 : [BlockingCoroutine{Active}@1f69e033, BlockingEventLoop@7e98c3d5]
일반함수 task 2 : Thread-27
일시중단함수 task 4 : Thread-27
일반함수 task 1 : Thread-27
일수중단함수 task 3 : Thread-27


## 코루틴과 스레드 처리 비교 

In [7]:
import java.util.concurrent.*

fun main216() = runBlocking {                          // 런블러킹 코루틴 작성       
    println(" 코루틴 처리 ")
    repeat(100_000) {                                  // 10만개의 코루틴 처리 작성 
        launch {
            delay(100L)
            if (it % 10000 == 0) {                     // 만개 처리후에 점표시 
                print(".")
            } 
        }
    }
}
main216() 

println()
fun main217() {                                         // 런블럭으로 코투틴 작성 
    println(" 스레드 처리")
    val executor = Executors.newFixedThreadPool(1024)   // 스레드 고유풀 생성 
    repeat(10000) {                                     // 만개만 처리 
        executor.submit {
            Thread.sleep(100L)
            if (it % 1000 == 0) {
                print("*")
            } 
        }
    }
    Thread.sleep(3000L)                                  // 서브 스페드 처리가 끝나도록 잠시 중단
    
    executor.shutdown()                                  // 풀 닫기 
}
main217() 

 코루틴 처리 
..........
 스레드 처리
**********